In [3]:
# # boto3 설치
!pip install boto3

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 12.6/12.6 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [1]:
import boto3

## Case 1 : No Credential Error

- 인증 오류를 해결하기 위해 AWS Access Key, Secret Access Key 입력 필요함.

In [7]:
# S3 클라이언트 생성
s3 = boto3.client("s3")

# 모든 버킷 목록 가져오기
response = s3.list_buckets()

# 버킷 이름 출력
for bucket in response["Buckets"]:
    print(bucket["Name"])

NoCredentialsError: Unable to locate credentials

## Case2 : dotenv

- 환경 변수 입력 및 보안을 위해 .env 파일에 기입하여 사용

In [6]:
!pip install python-dotenv

In [14]:
from dotenv import load_dotenv
import os

# 같은 폴더 내 .env 파일 정도 가져오기
load_dotenv()

# 각 변수에 .env 변수 load
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
REGION = os.getenv('region')


# # AWS S3 관련 설정

# S3_BUCKET = os.getenv('S3_BUCKET')

# AWS S3 클라이언트 생성
s3 = boto3.client("s3", aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

In [15]:
# 불러온 정보 확인
print("ACCESS_KEY : ", ACCESS_KEY[:-15] + ("*" * 15))
print("S3_REGION : ", REGION)

ACCESS_KEY :  AKIAU***************
S3_REGION :  ap-northeast-2


In [16]:
# 모든 버킷 목록 가져오기
response = s3.list_buckets()

# 버킷 이름 출력
for bucket in response["Buckets"]:
    print(bucket["Name"])

big9-example
big9-m9999
big9-test
big9-test-ara
big9-test-dongin
big9-test-ktk
big9-test-manhye
big9-test-seungyeon
bigdata9-ara
bigdata9-dongin
bigdata9-ktk
bigdata9-manhye
bigdata9-seungyeon
bigdata9-sooz
bigdata9-yewon
bigdata9-yura
dev-team-haejo-backup
dev-team-haejo-rawdata
fast-solmi2
fastapi-backup
jstest-4t
my--bucket-big9--12345
my--bucket-big9-kwaktaekyeong-75579
my--bucket-big9-seungyeon-big9-13579
my--bucket-big9-yewon-big9-32415
my--bucket-big9-yura-big9-1234
preprocessing-4t
raw-data-4t
result-4t
testoftestoftest


In [22]:
# S3 버킷 이름 정의
my_bucket_name = input("버킷의 이름을 입력하세요\n")
bucket_number = 12345

input_number = input(
    "버킷 고유 숫자를 입력하세요.\n미입력 시 기본 숫자인 12345로 지정됩니다.\n"
)
while True:
    if input_number == "":
        print("12345로 지정됩니다.")
        break
    else:
        try:
            number_check = int(input_number)
            if type(number_check) == int:
                bucket_number = number_check
                break
            else:
                print("숫자를 입력해주세요")
        except:
            print("숫자를 입력해주세요")


bucket_name = f"my--bucket-big9-{my_bucket_name}-{bucket_number}"

# S3 버킷 생성
s3.create_bucket(Bucket=bucket_name)

print(f"버킷 {bucket_name}이(가) 생성되었습니다.")

버킷 my--bucket-big8-mybucket-92923이(가) 생성되었습니다.


In [ ]:
# 생성 확인
# 모든 버킷 목록 가져오기
response = s3.list_buckets()

# 버킷 이름 출력
for bucket in response["Buckets"]:
    print(bucket["Name"])

## EC2 실행하기

In [13]:
ec2 = boto3.client(
    "ec2",
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)

NoRegionError: You must specify a region.

In [33]:
ec2_response = ec2.describe_instances()

# for instance in ec2_response['Name']

In [ ]:
ec2_response = ec2.describe_instances()

for reservation in ec2_response["Reservations"]:
    for instance in reservation["Instances"]:
        # 인스턴스의 Name 태그 찾기
        name = None
        if "Tags" in instance:
            for tag in instance["Tags"]:
                if tag["Key"] == "Name":
                    name = tag["Value"]
                    break

        print(f"Instance Name: {name if name else '---'}")
        # print(f"Instance ID: {instance['InstanceId']}")
        # print(f"Instance Type: {instance['InstanceType']}")
        # print(f"State: {instance['State']['Name']}")
        # print(f"Public IP: {instance.get('PublicIpAddress')}")
        # print(f"Launch Time: {instance['LaunchTime']}")
        print("----")

In [ ]:
# EC2 인스턴스 시작
Instance_Name = "server_test_dongin"  # 원하는 인스턴스 이름

response = ec2.run_instances(
    ImageId="ami-062cf18d655c0b1e8",
    InstanceType="t2.micro",  # 인스턴스 유형
    MinCount=1,  # 시작할 최소 인스턴스 수
    MaxCount=1,  # 시작할 최대 인스턴스 수
    KeyName="bigdata9_4",  # EC2 인스턴스에 접근할 키 페어 이름
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": f"{Instance_Name}"}],
        }
    ],
)

instance_id = response["Instances"][0]["InstanceId"]
print(f"새로운 EC2 인스턴스 {Instance_Name}이(가) 시작되었습니다.")
print(f"새로운 EC2 인스턴스 id 값은 {instance_id}입니다.")

In [61]:
import time

# 인스턴스의 Public IP 값을 확인
while True:
    ec2_response = ec2.describe_instances()

    for instance_ in ec2_response["Reservations"]:
        if instance_["Instances"][0].get("InstanceId") == instance_id:
            try:
                public_ip = instance_["Instances"][0].get("PublicIpAdress")
            except:
                pass
        else:
            pass

    if public_ip:
        print(f"새로운 EC2 인스턴스의 Public IP는 {public_ip}입니다.")
        break
    else:
        print("Public IP가 아직 할당되지 않았습니다. 잠시 기다려 주세요.")
        time.sleep(10)

In [89]:
# 인스턴스 종료하기
ec2.terminate_instances(InstanceIds=[instance_id])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-000ace9269fd19733',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'afb0fcaf-96af-4fe0-90f5-2b175bbd7c32',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'afb0fcaf-96af-4fe0-90f5-2b175bbd7c32',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '426',
   'date': 'Sun, 18 Aug 2024 14:54:09 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}